# making hyperstacks from zm01 recordings by iterating through a pandas with its dimensions
since there are issuses with reding metadata form the tiff files I calculated and stored number of volumes, z number height and width of each recording in a dataframe.
https://forum.image.sc/t/ome-tiff-z-stack-to-bigtiff/50964/7

In [2]:
import tifffile as tiff
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re
from imutils.src import imfunctions
import os
from natsort import natsorted
import math
import glob

Using TensorFlow backend.


## create dataframe with metadata

### get the filenames of recordings

In [4]:
#getting file names
path='/scratch/zimmer/DanielM/zim01/zim_443_urx/ZIm443/flurescence_btiffs/cam1/'
for file in natsorted(os.listdir(path)):
    if file.endswith('.btf'):
        before,sep,after=file.partition('-channel')
        print(before)

2021-07-15_14-27-22_zim443_wor2_on
2021-07-15_15-12-30_zim443_wor3_on
2021-07-15_16-49-36_zim443_wor4_on
2021-07-15_17-14-13_zim443_wor5_off
2021-07-15_17-48-38_zim443_wor6_off


### read in a csv with filenames, number of z steps, height and width (all written down manually)

In [ ]:
filename='/groups/zimmer/Daniel_Mitic/code/imutils/imutils/zim443_dimensions.csv'
df=pd.read_csv(filename,sep=';')

## get the amount of volumes

In [7]:
path='/scratch/zimmer/DanielM/zim01/zim_443_urx/ZIm443/flurescence_btiffs/cam1/'

In [8]:
#volumes=len of pages/z_stacks
#some of them are not integers: if z is an uneven number the last volume is not gonna have a full amount of stacks. this is why i round it up with math(ceil)
for i,file in enumerate(natsorted(os.listdir(path))):
    if 'camera' in file:
        with tiff.TiffFile(os.path.join(path,file)) as tif:
            print(math.ceil(len(tif.pages)/df.iloc[i,2]))

NameError: name 'df' is not defined

In [ ]:
#i inserted the number of volumes manually in the csv, so i read it in again here
filename='/groups/zimmer/Daniel_Mitic/code/imutils/imutils/zim443_dimensions.csv'
df=pd.read_csv(filename,sep=';')

# create the hyperstacks

In [ ]:
path='/scratch/zimmer/DanielM/ZIm443/cam1/' #path with the folders containing the ome.tif
output_file='/scratch/zimmer/DanielM/hyperstack_daniel/'
dtype='uint16'
imagej=True,
metadata={'axes': 'TZYX'}

In [ ]:
for i,file in enumerate(natsorted(glob.glob(path+'*camera*'))):
    #grab the name of the file
    before,sep,name=file.partition('cam1/')
    
    #making sure the output files have different names
    output=os.path.join(output_file+name+'.btf')
    
    #specify the input path as the subfolders which contain the ome.tifs
    input_path=os.path.join(path+name+'/') 
    
    #iterate through the dataframe to get the corresponding shape for each recording
    shape=tuple(df.iloc[i,1:5])
    
    #make hyperstacks
    imfunctions.make_hyperstack_from_ometif(input_path,output,shape,dtype,imagej,metadata)
print('done')